In [123]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from numpy.linalg import lstsq
from itertools import combinations
from statsmodels.api import OLS
from sklearn.linear_model import LassoCV, RidgeCV
#from tqdm import tqdm
import statsmodels.api as sm
from scipy import stats

In [252]:
## Fetching data - using value weighted returns

industries = pd.read_csv("data/17_Industry_Portfolios.csv",index_col="Date")
industries.index = pd.to_datetime(industries.index,format="%Y%m")
industries = industries.resample("M").last()
industries = industries["1927-01-31":]

mom_size = pd.read_csv("data/MOM-size.csv",index_col="Date")
mom_size.index = pd.to_datetime(mom_size.index,format="%Y%m")
mom_size = mom_size.resample("M").last()
mom_size = mom_size["1927-01-31":]

size_value = pd.read_csv("data/size_value.csv",index_col="Date")
size_value.index = pd.to_datetime(size_value.index,format="%Y%m")
size_value = size_value.resample("M").last()
size_value = size_value["1927-01-31":]


#momentum = pdr.get_data_famafrench("F-F_Momentum_Factor_daily", start="1920", end="2020-12-31")[0]
#momentum = momentum.resample("M").last().squeeze()
#momentum.rename(level=0,index="Mom")

momentum = pd.read_csv("data/F-F_Momentum_Factor.csv",index_col="Date")
momentum.index = pd.to_datetime(momentum.index,format="%Y%m")
momentum = momentum.resample("M").last().squeeze()

# Value weighted market

vwm = pd.read_csv("data/VWM.csv",index_col="Date")
vwm.index = pd.to_datetime(vwm.index, format="%Y%m")
vwm = vwm.resample("M").last()
vwm = vwm["1927-01-31":]


In [220]:
hml = 0.5 * (size_value["SMALL HiBM"] + size_value["BIG HiBM"]) - 0.5 * (size_value["SMALL LoBM"] + size_value["BIG LoBM"] )
mom = 0.5 * (mom_size["SMALL HiPRIOR"] + mom_size["BIG HiPRIOR"]) - 0.5 * (mom_size["SMALL LoPRIOR"] + mom_size["BIG LoPRIOR"])

In [221]:
## Cross Validation function

def xval_5fold(y, x , random=False, seed = 20201231,fold=5):
    # Use numpy arrays for simplicity
    y = np.asarray(y)
    x = np.asarray(x)
    n = y.shape[0]
    if random:
        # If randomisation is needed, use either the default or provided seed
        rg = np.random.default_rng(seed)
        # Generate a set of index values to use to randomly reorder the data
        # After randomisation, we can use the data as if it is is inorder!
        ind = rg.permutation(np.arange(n))
        y = y[ind]
        x = x[ind]
    # Compute the block size
    block = n / fold
    sse = 0.0
    for i in range(int(fold)):
        # Start and end of each block need to be integers or we lose an observation
        # Rounding ensures that we get all observations since int rounds down
        st = int(np.round(i * block))
        en = int(np.round((i + 1) * block))
        # Construct the indicies of the observations that we leave out
        leave_out = np.r_[st:en]
        # The included are the one that we don't leave out
        include = np.setdiff1d(np.arange(n), leave_out)
        # Compute the regression coefficients
        beta = lstsq(x[include], y[include], rcond=None)[0]
        # Compute the residuals and add to the sse
        resid = y[st:en]- x[st:en] @ beta
        sse += resid @ resid
    return sse

# Randomisation is better when we suspect the model may not be totally stable

In [506]:
# forward stepwise function: 
# requires x: x dataframe , y series, p = number of columns, requires xval function too
# input x, y, p
def forward_stepwise(x, y, p,random=False,fold=5):
    included = []

    for i in range(p):
        excluded = [col for col in x if col not in included]
        best_sse = np.inf
        for col in excluded:
            try_x = x[included + [col]]
            beta = lstsq(try_x, y, rcond=None)[0]
            resid = y - try_x @ beta
            sse = resid @ resid
            if sse < best_sse:
                best_sse = sse
                next_var = col
        included.append(next_var)
    
    fsr_sse_sv = {}
    for i in range(1,p+1):
        fsr_sse_sv[i] = xval_5fold(y,x[included[:i]],random=random, fold=fold)
    fsr_sse_sv = pd.Series(fsr_sse_sv)
    forward_step_model = included[:fsr_sse_sv.idxmin()]
    return forward_step_model

In [500]:
# Hybrid stepwise function
# requires x: x dataframe , y series, p = number of columns, requires xval function too
# input x, y, p
def hybrid_stepwise(x, y, p,random=False,fold=5):
    included = []

    for i in range(p):
        excluded = [col for col in x if col not in included]
        best_sse = np.inf
        for col in excluded:
            try_x = x[included + [col]]
            beta = lstsq(try_x, y, rcond=None)[0]
            resid = y - try_x @ beta
            sse = resid @ resid
            if sse < best_sse:
                best_sse = sse
                next_var = col
        included.append(next_var)

    end_models = {}
    for i in range(1,p+1):
        x_col = included[:i]
        regressors = x[x_col]
        end_models[i] = back_stepwise(regressors,y,p,random=random,fold=fold)

    hsr_sse_sv = {}

    for i in range(1,p+1):
        hsr_sse_sv[i] = xval_5fold(y,x[end_models[i]],random=random, fold=fold)
    hsr_sse_sv = pd.Series(hsr_sse_sv)
    hybrid_step_model = end_models[hsr_sse_sv.idxmin()]


    return hybrid_step_model

In [496]:
# Backward stepwise components

def back_stepwise(x,y,p,random=False,fold=5):


    included = list(x.columns)
    
    removed = []
    p = x.shape[1]

    for i in range(p):
        best_sse = np.inf
        for col in included:
            try_col = included[:]
            try_col.remove(col)
            try_x = x[try_col]
            beta = lstsq(try_x,y,rcond=None)[0]
            resid = y - try_x @ beta
            sse = resid @ resid
            if sse < best_sse:
                best_sse = sse
                next_drop = col
        removed.append(next_drop)
        included.remove(next_drop)

    #print(removed)

    included = removed[::-1]

    bsr_sse_sv = {}
    for i in range(1,p+1):
        bsr_sse_sv[i] = xval_5fold(y,x[included[:i]],random=random,fold=fold)
    bsr_sse_sv = pd.Series(bsr_sse_sv)
    backstep_model = included[:bsr_sse_sv.idxmin()]
    return backstep_model

In [223]:
### LASSO function

def lasso_reg(x,y):

    x_scale = x.std(ddof=0)
    y_scale = y.std(ddof=0)
    std_x = x / x_scale
    std_y = y / y_scale
    std_x = std_x

    lasso_cv = LassoCV(fit_intercept=False)
    lasso_cv = lasso_cv.fit(std_x,std_y)
    #print(f"Optimal Alpha = {lasso_cv.alpha_}")
    lasso_beta = lasso_cv.coef_  * (y_scale / x_scale)
    return lasso_beta


In [224]:
# General to Specific selection

def gts(x,y):
    cv = stats.norm.ppf(.995)

    included  = list(x.columns)
    y = y
    for i in range(16):
        x = x.loc[:,included]
        res = sm.OLS(y,x).fit(cov_type="HC0")
        tstats = res.tvalues
        if tstats.abs().min() < cv:
            sorted_tstats = tstats.abs().sort_values()
            remove = sorted_tstats.index[0]
            included.remove(remove)
        else:
            break
    return included

In [225]:
# Ridge regression 

def ridge_reg(x,y):
    x_scale = x.std(ddof=0)
    y_scale = y.std(ddof=0)
    std_x = x / x_scale
    std_y = y / y_scale

    ridge_cv = RidgeCV(fit_intercept=False, alphas=np.linspace(1, 100, 100))
    ridge_cv = ridge_cv.fit(std_x, std_y)
    #print(f"Optimal alpha = {ridge_cv.alpha_}")
    start = ridge_cv.alpha_ - (1/5 * ridge_cv.alpha_)
    end = ridge_cv.alpha_ + (1/5 * ridge_cv.alpha_)

    ridge_cv = RidgeCV(fit_intercept=False, alphas=np.linspace(start, end, 2001))
    ridge_cv = ridge_cv.fit(std_x, std_y)
    #print(f"Optimal alpha = {ridge_cv.alpha_}")
    return ridge_cv.coef_ * (y_scale / x_scale)


HML (Value) and momentum,  rolling 
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [519]:
ridge_reg(x,y)

Food     0.101894
Mines    0.003133
Oil      0.154178
Clths   -0.049410
Durbl    0.008469
Chems    0.056866
Cnsum    0.126822
Cnstr    0.075997
Steel    0.009025
FabPr    0.011794
Machn    0.140009
Cars     0.012917
Trans    0.036484
Utils    0.032113
Rtail    0.093233
Finan    0.002801
Other    0.203383
dtype: float64

In [545]:

hml = 0.5 * (size_value["SMALL HiBM"] + size_value["BIG HiBM"]) - 0.5 * (size_value["SMALL LoBM"] + size_value["BIG LoBM"] )
mom = 0.5 * (mom_size["SMALL HiPRIOR"] + mom_size["BIG HiPRIOR"]) - 0.5 * (mom_size["SMALL LoPRIOR"] - mom_size["BIG LoPRIOR"])

"""Toggle to select the number of years data trained on, 20, 10 or 5"""


nobs = industries.shape[0]

# rolling regressions
for i in range(240,nobs-60,12): 

    # Toggle to select number of years
    #years = 12 * years

    # Select the in sample training data
    y = mom.iloc[i-240:i]                 # ----  change to mom or hml as desired   
    y = y.squeeze()
    x = industries.iloc[i-240:i]         
    t, p = x.shape
    
    # Form the out of sample data
    y_oos = (mom.iloc[i:i+60]).squeeze()    # ----  change to mom or hml as desired
    x_oos = industries.iloc[i:i+60]

    #---------------------------------------------------------------------------------------------------------
    # Simple OLS Regression
    
    res = sm.OLS(y,x).fit(cov_type="HC0")
    pred_ols = x_oos @ res.params
    resid_oos = y_oos - pred_ols
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Simple OLS")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    #print()
    

    #---------------------------------------------------------------------------------------------------------
    # Hybrid Stepwise
    """
    model = hybrid_stepwise(x, y, p)
    res = sm.OLS(y,x[model]).fit(cov_type="HC0")
    
    pred_hy = x_oos[model] @ res.params
    resid_oos = y_oos - pred_hy
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Hybrid Stepwise")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    #print()
    """

      #---------------------------------------------------------------------------------------------------------
    # Forward Stepwise
    """
    model = forward_stepwise(x, y, p)
    res = sm.OLS(y,x[model]).fit(cov_type="HC0")
    
    pred_fwd = x_oos[model] @ res.params
    resid_oos = y_oos - pred_fwd
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Forward Stepwise")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    #print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Lasso on the forward stepwise
    """
    beta = lasso_reg(x[model],y)
    pred_fwd_lasso = x_oos[model] @ beta
    resid_oos = y_oos - pred_fwd_lasso
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Lasso on Forward Stepwise")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Lasso regression on all industries
    """
    beta = lasso_reg(x,y)
    pred_lasso = x_oos @ beta
    resid_oos = y_oos - pred_lasso
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Lasso")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

     #---------------------------------------------------------------------------------------------------------
    # Ridge selection on all industries
    """
    beta = ridge_reg(x,y)
    pred_ridge = x_oos @ beta
    resid_oos = y_oos - pred_ridge
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Ridge")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # General to Specific selection
    """
    model = gts(x,y)
    res = sm.OLS(y,x[model]).fit(cov_type="HC0")
    pred_gts = x_oos[model] @ res.params
    resid_oos = y_oos - pred_gts
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("General to Specific")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Lasso on general to specific
    """
    beta = lasso_reg(x[model],y)
    pred_gts_lasso = x_oos[model] @ beta
    resid_oos = y_oos - pred_gts_lasso
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    #print("Lasso on General to Specific")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Ridge on general to specific
    """
    beta = ridge_reg(x[model],y)
    pred_gts_ridge = x_oos[model] @ beta
    resid_oos = y_oos - pred_gts_ridge
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Ridge on General to Specific")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Mixed model
    """
    pred_mixed = (1/2)* pred_gts_ridge + (1/2) * pred_fwd_lasso #+ (1/3)*pred_ridge
    resid_oos = y_oos - pred_mixed
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """
    
    """
    print("Naive Averaging")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """





Simple OLS
174.61834514672313
0.8575048554063102
Simple OLS
181.9502008709353
0.8630329751260515
Simple OLS
130.50919840029206
0.8594737799647562
Simple OLS
135.32832785023004
0.8687819369164445
Simple OLS
131.6814824214207
0.8637004150409984
Simple OLS
91.85863178917154
0.9027620969146681
Simple OLS
74.81012506152226
0.9205178485005661
Simple OLS
94.58640731374952
0.9052951216186393
Simple OLS
96.3183281902895
0.8904481165065724
Simple OLS
103.62942895691276
0.8913336350972033
Simple OLS
121.04478442798424
0.8537695725004106
Simple OLS
162.80946176364836
0.8294525979483875
Simple OLS
138.1742940530358
0.8599635396066805
Simple OLS
150.2666370413139
0.8326125195077404
Simple OLS
157.14554502535378
0.8167952138239376
Simple OLS
201.2224762401865
0.7888320463415852
Simple OLS
205.47089998758446
0.7803471242269802
Simple OLS
249.82725181296223
0.7782162892903848
Simple OLS
278.492613845791
0.7848172581863917
Simple OLS
222.36138089241078
0.8540615144076289
Simple OLS
172.16806692520788
0.

HML (Value) and Momentum, positive side
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [546]:
hml = 0.5 * (size_value["SMALL HiBM"] + size_value["BIG HiBM"]) 
mom = 0.5 * (mom_size["SMALL HiPRIOR"] + mom_size["BIG HiPRIOR"])

"""Toggle to select the number of years data trained on, 20, 10 or 5"""
#years = 20

nobs = industries.shape[0]
# rolling regressions
for i in range(240,nobs-60,12): 

    # Toggle to select number of years
    #years = 12 * years

    # Select the in sample training data
    y = mom.iloc[i-60:i]                   # ----  change to mom or hml as desired
    y = y.squeeze()
    x = industries.iloc[i-60:i]         
    t, p = x.shape
    
    # Form the out of sample data
    y_oos = (mom.iloc[i:i+60]).squeeze()      # ----  change to mom or hml as desired
    x_oos = industries.iloc[i:i+60]

    #---------------------------------------------------------------------------------------------------------
    # Simple OLS Regression
    """
    res = sm.OLS(y,x).fit(cov_type="HC0")
    pred_ols = x_oos @ res.params
    resid_oos = y_oos - pred_ols
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Simple OLS")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Forward Stepwise
    """
    model = forward_stepwise(x, y, p)
    res = sm.OLS(y,x[model]).fit(cov_type="HC0")
    
    pred_fwd = x_oos[model] @ res.params
    resid_oos = y_oos - pred_fwd
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Forward Stepwise")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Lasso on the forward stepwise
    """
    beta = lasso_reg(x[model],y)
    pred_fwd_lasso = x_oos[model] @ beta
    resid_oos = y_oos - pred_fwd_lasso
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Lasso on Forward Stepwise")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Lasso regression on all industries
    
    beta = lasso_reg(x,y)
    pred_lasso = x_oos @ beta
    resid_oos = y_oos - pred_lasso
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Lasso")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    

     #---------------------------------------------------------------------------------------------------------
    # Ridge selection on all industries
    """
    beta = ridge_reg(x,y)
    pred_ridge = x_oos @ beta
    resid_oos = y_oos - pred_ridge
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Ridge")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # General to Specific selection
    """
    model = gts(x,y)
    res = sm.OLS(y,x[model]).fit(cov_type="HC0")
    pred_gts = x_oos[model] @ res.params
    resid_oos = y_oos - pred_gts
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("General to Specific")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Lasso on general to specific
    """
    beta = lasso_reg(x[model],y)
    pred_gts_lasso = x_oos[model] @ beta
    resid_oos = y_oos - pred_gts_lasso
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    #print("Lasso on General to Specific")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Ridge on general to specific
    """
    beta = ridge_reg(x[model],y)
    pred_gts_ridge = x_oos[model] @ beta
    resid_oos = y_oos - pred_gts_ridge
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Ridge on General to Specific")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Mixed model
    """
    pred_mixed = (1/2)* pred_gts_ridge + (1/2) * pred_fwd_lasso #+ (1/3)*pred_ridge
    resid_oos = y_oos - pred_mixed
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """
    
    """
    print("Naive Averaging")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """



Lasso
133.44575353110713
0.9061673260462692

Lasso
67.66678729663623
0.9543554034311761

Lasso
58.348872591721175
0.9445368395931908

Lasso
118.0322507770552
0.900068205322475

Lasso
75.18960903515554
0.9264927657368716

Lasso
80.03108582722555
0.9121044439098895

Lasso
94.07790937060011
0.8957515174548754

Lasso
107.65511543657618
0.8928539355157883

Lasso
64.31348313955803
0.9214770990862876

Lasso
81.41057123669056
0.9095065314553851

Lasso
115.06226927394765
0.8707467661363569

Lasso
134.90970627336185
0.8762827887848084

Lasso
122.11916994218693
0.8851747293247001

Lasso
91.15249451574007
0.907739306943543

Lasso
124.30732180195179
0.8823386167073944

Lasso
259.2534007426769
0.78796392216233

Lasso
348.96744074378364
0.7312770639736086

Lasso
359.101546392073
0.7804589791312682

Lasso
419.2834647427571
0.7848852371645252

Lasso
302.8261118050366
0.8601517735929434

Lasso
223.1573101378136
0.8948360476233538

Lasso
192.21548453391352
0.8921896338509654

Lasso
306.64144342931814
0.8

HML (Value) and Momentum, negative side
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
hml = - 0.5 * (size_value["SMALL LoBM"] + size_value["BIG LoBM"] )
mom = - 0.5 * (mom_size["SMALL LoPRIOR"] - mom_size["BIG LoPRIOR"])

In [550]:
hml = - 0.5 * (size_value["SMALL LoBM"] + size_value["BIG LoBM"] )
mom = - 0.5 * (mom_size["SMALL LoPRIOR"] - mom_size["BIG LoPRIOR"])

"""Toggle to select the number of years data trained on, 20, 10 or 5"""
#years = 10

nobs = hml.shape[0]
# rolling regressions
for i in range(240,nobs-60,12): 

    # Toggle to select number of years
    #years = 12 * years

    # Select the in sample training data
    y = mom.iloc[i-60:i]                  # ----  change to mom or hml as desired
    y = y.squeeze()
    x = industries.iloc[i-60:i]         
    t, p = x.shape
    
    # Form the out of sample data
    y_oos = (mom.iloc[i:i+60]).squeeze()     # ----  change to mom or hml as desired
    x_oos = industries.iloc[i:i+60]

    #---------------------------------------------------------------------------------------------------------
    # Simple OLS Regression
    """
    res = sm.OLS(y,x).fit(cov_type="HC0")
    pred_ols = x_oos @ res.params
    resid_oos = y_oos - pred_ols
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Simple OLS")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Forward Stepwise
    
    model = forward_stepwise(x, y, p)
    res = sm.OLS(y,x[model]).fit(cov_type="HC0")
    
    pred_fwd = x_oos[model] @ res.params
    resid_oos = y_oos - pred_fwd
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Forward Stepwise")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    

    #---------------------------------------------------------------------------------------------------------
    # Lasso on the forward stepwise
    """
    beta = lasso_reg(x[model],y)
    pred_fwd_lasso = x_oos[model] @ beta
    resid_oos = y_oos - pred_fwd_lasso
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Lasso on Forward Stepwise")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Lasso regression on all industries
    """
    beta = lasso_reg(x,y)
    pred_lasso = x_oos @ beta
    resid_oos = y_oos - pred_lasso
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Lasso")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

     #---------------------------------------------------------------------------------------------------------
    # Ridge selection on all industries
    """
    beta = ridge_reg(x,y)
    pred_ridge = x_oos @ beta
    resid_oos = y_oos - pred_ridge
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Ridge")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # General to Specific selection
    """
    model = gts(x,y)
    res = sm.OLS(y,x[model]).fit(cov_type="HC0")
    pred_gts = x_oos[model] @ res.params
    resid_oos = y_oos - pred_gts
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("General to Specific")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Lasso on general to specific
    """
    beta = lasso_reg(x[model],y)
    pred_gts_lasso = x_oos[model] @ beta
    resid_oos = y_oos - pred_gts_lasso
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    #print("Lasso on General to Specific")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Ridge on general to specific
    """
    beta = ridge_reg(x[model],y)
    pred_gts_ridge = x_oos[model] @ beta
    resid_oos = y_oos - pred_gts_ridge
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """

    """
    print("Ridge on General to Specific")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """

    #---------------------------------------------------------------------------------------------------------
    # Mixed model
    """
    pred_mixed = (1/2)* pred_gts_ridge + (1/2) * pred_fwd_lasso #+ (1/3)*pred_ridge
    resid_oos = y_oos - pred_mixed
    oos_sse = resid_oos @ resid_oos
    oos_tss = (y_oos **2).sum()
    oos_r2 = 1- oos_sse/ oos_tss
    """
    
    """
    print("Naive Averaging")
    print(oos_sse)
    #print(oos_tss)
    print(oos_r2)
    #print(model)
    print()
    """



Forward Stepwise
91.02549210759639
-0.35440946403984386

Forward Stepwise
68.77261224457581
-0.07245856886839497

Forward Stepwise
80.02692181430633
-0.23032644284010395

Forward Stepwise
80.82956677552627
-0.04523773564481237

Forward Stepwise
38.66112309886061
0.28943449379448405

Forward Stepwise
44.79575705588002
0.16317867069400138

Forward Stepwise
50.1224599624411
0.2441157181763589

Forward Stepwise
57.55880941758603
0.17827656622732435

Forward Stepwise
36.95667262741768
0.3508942670791092

Forward Stepwise
49.081431074867076
0.30448576490707147

Forward Stepwise
62.361515122569294
0.25637912626587556

Forward Stepwise
57.76597532906905
0.33956551622897246

Forward Stepwise
64.14617760041361
0.20910492713944495

Forward Stepwise
60.140584409182104
0.22909288021275886

Forward Stepwise
61.74454126343561
0.2543582685883713

Forward Stepwise
72.03611845070678
0.28842569084947733

Forward Stepwise
72.80564338101209
0.4081064563140443

Forward Stepwise
67.54912963157335
0.526174515

--------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------
START: momentum 
---------------------------